In [1]:
import numpy as np, matplotlib.pyplot as plt, os, pickle
import tools21cm as t2c, py21cmfast as p21c
from py21cmfast import plotting
import random, zipfile
from astropy.cosmology import FlatLambdaCDM
from tqdm import tqdm

from utils_network.prediction import SegUnet21cmPredict
from utils.other_utils import RescaleData

import matplotlib.gridspec as gridspec

from itertools import permutations
from datetime import datetime

outpath = './'

In [2]:
from tensorflow.keras.models import load_model
from utils_network.metrics import iou, iou_loss, dice_coef, dice_coef_loss, balanced_cross_entropy, phi_coef
from config.net_config import NetworkConfig

def LoadSegUnetModel(cfile):
    avail_metrics = {'binary_accuracy':'binary_accuracy', 'iou':iou, 'dice_coef':dice_coef, 'iou_loss':iou_loss, 'dice_coef_loss':dice_coef_loss, 'phi_coef':phi_coef, 'mse':'mse', 'mae':'mae', 'binary_crossentropy':'binary_crossentropy', 'balanced_cross_entropy':balanced_cross_entropy}
    conf = NetworkConfig(conf_file)

    PATH_OUT = conf.resume_path
    MODEL_EPOCH = conf.best_epoch
    METRICS = [avail_metrics[m] for m in np.append(conf.loss, conf.metrics)]
    cb = {func.__name__:func for func in METRICS if not isinstance(func, str)}
    model_loaded = load_model('%smodel-sem21cm_ep%d.h5' %(PATH_OUT+'checkpoints/', MODEL_EPOCH), custom_objects=cb)
    
    print(' Loaded model:\n %smodel-sem21cm_ep%d.h5' %(conf.resume_path, MODEL_EPOCH))
    return model_loaded

conf_file = '/home/michele/Documents/PhD_Sussex/output/ML/SegNet/tests/runs/net2D_021020.ini'
model = LoadSegUnetModel(conf_file)

Using TensorFlow backend.


Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 Loaded model:
 /home/michele/Documents/PhD_Sussex/output/ML/dataset/outputs/02-10T23-52-36_128slice/model-sem21cm_ep56.h5


In [14]:
#xHI = np.array([0.2, 0.5, 0.8])
#redshift = np.array([7.310, 8.032, 8.720])
z = 7.310
zeta = 39.204
Rmfp = 12.861
Tvir = 4.539

params = {'HII_DIM':128, 'DIM':384, 'BOX_LEN':256}
my_ext = [0, params['BOX_LEN'], 0, params['BOX_LEN']]

c_params = {'OMm':0.27, 'OMb':0.046, 'SIGMA_8':0.82, 'POWER_INDEX':0.96}
a_params = {'HII_EFF_FACTOR':zeta, 'R_BUBBLE_MAX':Rmfp, 'ION_Tvir_MIN':Tvir}

ic = p21c.initial_conditions(user_params=params, cosmo_params=c_params, random_seed=2021)
cube = p21c.run_coeval(redshift=z, init_box=ic, astro_params=a_params, zprime_step_factor=1.05)

dT = cube.brightness_temp
xH = cube.xH_box

uvfile = '/home/michele/Documents/PhD_Sussex/output/ML/dataset/inputs/uvmap_128_z7-20.pkl'
if not (os.path.exists(uvfile)):
    print('uv-map pickle not found')
else:
    uvs = pickle.load(open(uvfile, 'rb'))

uv = uvs['%.3f' %z]
Nant = uvs['Nant']



In [ ]:
dT1 = t2c.subtract_mean_signal(dT, los_axis=2)
tobs = 500

for i in range(10):
    str_seed = [var for var in datetime.now().strftime('%d%H%M%S')]
    np.random.shuffle(str_seed)
    seed = int(''.join(str_seed))

    np.random.seed(seed)
    noise_cube = t2c.noise_cube_coeval(params['HII_DIM'], z, obs_time=tobs, boxsize=params['BOX_LEN'], uv_map=uv, N_ant=Nant, verbose=False)

    dT3 = t2c.smooth_coeval(dT1+noise_cube, z, box_size_mpc=params['HII_DIM'], max_baseline=2.0, ratio=1.0, nu_axis=2)

    X_tta = SegUnet21cmPredict(unet=model, x=dT3, TTA=False)
    X_seg = np.round(np.mean(X_tta, axis=0))

    mfp_pred_ml = t2c.bubble_stats.mfp(X_seg, xth=0.5, boxsize=params['BOX_LEN'], iterations=2000000, verbose=False, upper_lim=False, bins=None, r_min=None, r_max=None)

    plt.loglog(mfp_pred_ml[0], mfp_pred_ml[1], '-', color='tab:blue', label=r'SegUNet', lw=2)
plt.savefig('tobs%d_0.5.png' %tobs)
plt.show(), plt.clf()

Creating the noise cube...


100%|██████████| 128/128 [00:00<00:00, 240.41it/s]


...noise cube created.


100%|██████████| 1/1 [00:50<00:00, 50.13s/it]


MFP method applied on 3D data (ver 1.0)
Completed: 100.0%
Program runtime: 0.355505 minutes.
The output contains a tuple with three values: r, rdP/dr
The curve has been normalized.
Creating the noise cube...


...

MFP method applied on 3D data (ver 1.0)
Completed: 100.0%
Program runtime: 0.268089 minutes.
The output contains a tuple with three values: r, rdP/dr
The curve has been normalized.
Creating the noise cube...


100%|██████████| 128/128 [00:00<00:00, 245.66it/s]


...noise cube created.


100%|██████████| 1/1 [00:39<00:00, 39.97s/it]


MFP method applied on 3D data (ver 1.0)
